In [99]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime as DT
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, BatchNormalization,Dropout
from tensorflow.keras import optimizers
import os
from matplotlib import pyplot as plt
from sklearn import svm, datasets
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [44]:
patients = pd.read_csv('./maindata/patients.csv')
allergies = pd.read_csv('./maindata/allergies.csv')

In [45]:
allergies = allergies[allergies.CATEGORY != 'medication']
print(len(allergies['CODE'].unique()))

15


In [46]:
AllergiesDetails = allergies[['CODE','DESCRIPTION','CATEGORY','REACTION1','DESCRIPTION1','SEVERITY1','REACTION2','DESCRIPTION2','SEVERITY2']]
AllergiesDetails = AllergiesDetails.drop_duplicates(subset='CODE',keep='last')

In [47]:
AllergiesDetails.to_csv('AllergiesList.csv',index=False)

In [48]:
allergies = allergies[['PATIENT','CODE','START']]

In [49]:
allergies = pd.merge(allergies,patients,how='left',left_on='PATIENT',right_on='Id')

In [50]:
allergies=allergies[['START','BIRTHDATE','LAT','LON','CODE']]
allergies['START'] = pd.to_datetime(allergies['START'], errors='coerce')
allergies['BIRTHDATE'] = pd.to_datetime(allergies['BIRTHDATE'], errors='coerce')

In [51]:
allergies['AGE'] = (allergies['START'] - allergies['BIRTHDATE']).astype('<m8[Y]')

In [52]:
allergies=allergies[['AGE','LAT','LON','CODE']]

In [53]:
allergies.to_csv('CleanedData.csv')

In [54]:
allergies.head()

,AGE,LAT,LON,CODE
0,1.0,41.930492,-71.003337,288328004
1,1.0,41.930492,-71.003337,84489001
2,1.0,41.930492,-71.003337,260147004
3,1.0,41.930492,-71.003337,264287008
4,1.0,41.930492,-71.003337,782576004


In [55]:
allergies_label = allergies.pop('CODE')

In [56]:
allergies_labels = pd.get_dummies(allergies_label,prefix="allergy")

In [57]:
allergies = pd.concat([allergies,allergies_labels],axis=1)

## Splitting Data in training and test

In [58]:
complete_train_dataset, complete_test_dataset = train_test_split(allergies,test_size=0.1)
# complete_test_dataset,complete_valid_dataset = train_test_split(temp_test_dataset,test_size=0.5)

In [59]:
train_dataset = complete_train_dataset.iloc[:,0:3]
train_labels = complete_train_dataset.iloc[:,3:]

In [60]:
test_dataset = complete_test_dataset.iloc[:,0:3]
test_labels = complete_test_dataset.iloc[:,3:]

#### Normalizing data

In [61]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()


In [62]:
def norm(x):
    return (x-train_stats['mean'])/train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


In [63]:
print(normed_test_data.shape[1])
print(train_labels.shape[1])

3
15


In [140]:

# model = Pipeline(steps=[('svr',SVR()),('lr_multi',MultiOutputClassifier(LogisticRegression()))])
# # model = MultiOutputRegressor(svr)
# model = model.fit(train_dataset,train_labels)
                        
# Create the SVR regressor
# svr = SVR(epsilon=0.2)

# Create the Multioutput Regressor
# model = MultiOutputRegressor(svr)

model = Pipeline([('reg', MultiOutputRegressor(SVR()))])


grid_param_svr = {
    'reg__estimator__C': [0.01,0.1,1,10,100,1000],
    'reg__estimator__epsilon':[0.1,0.2,0.3,0.4,0.5],
    'reg__estimator__degree':[2,3,4,5,6,7]
}

gs_svr = (GridSearchCV( estimator=model, 
                        param_grid=grid_param_svr, 
                        cv=20,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = -1,
                        refit=True)
         )

gs_svr = gs_svr.fit(train_dataset,train_labels)


In [133]:
print(gs_svr.cv_results_) 

{'mean_fit_time': array([2.41894126, 2.36216817, 2.39213607, 2.40861759, 2.46942134,
       2.52347138, 2.58893754, 2.62860737, 2.65331991, 2.69545245,
       2.74312973, 2.79938626, 2.81064675, 2.8382699 , 2.96860433,
       2.93681364, 3.02814488, 3.01947212, 3.10003173, 3.12237501,
       3.13633964, 3.22752037, 3.12314875, 3.1592957 , 3.17418509,
       3.18316083, 3.20041938, 3.26049755, 3.24754624, 3.25905926,
       3.2789597 , 3.3242506 , 3.33163385, 3.32380729, 3.32274685,
       3.26819751]), 'std_fit_time': array([0.0586702 , 0.07250647, 0.05004341, 0.05243535, 0.02818597,
       0.04559466, 0.0299417 , 0.01878593, 0.02765444, 0.03266924,
       0.03928588, 0.04621493, 0.04146611, 0.03298097, 0.08409783,
       0.05482811, 0.04848063, 0.03134217, 0.08320375, 0.04799239,
       0.09630751, 0.08163775, 0.03508144, 0.04249754, 0.04536519,
       0.0226225 , 0.02591203, 0.04568118, 0.02865255, 0.0290122 ,
       0.03527281, 0.03042337, 0.02920709, 0.02711861, 0.04040156,
       

In [141]:
print(gs_svr.best_estimator_) 

Pipeline(steps=[('reg', MultiOutputRegressor(estimator=SVR(C=1000, degree=2)))])


In [142]:
accuracy=gs_svr.best_score_
print(accuracy)

-0.06322496977572192


In [143]:
example_batch = normed_test_data
example_result = gs_svr.predict(example_batch)
example_result

array([[1.29588949, 0.27096344, 0.9208856 , ..., 0.88478091, 1.10488456,
        1.10562397],
       [1.29249602, 0.26701487, 0.91138042, ..., 0.86499598, 1.09238822,
        1.10089911],
       [1.29442838, 0.26832491, 0.91748929, ..., 0.87188452, 1.09816425,
        1.10047468],
       ...,
       [1.30263319, 0.27433818, 0.92403025, ..., 0.86766847, 1.10269673,
        1.10497743],
       [1.29823302, 0.27244142, 0.9222273 , ..., 0.88220838, 1.10550973,
        1.1067512 ],
       [1.29072276, 0.26632545, 0.91639972, ..., 0.87885852, 1.09864222,
        1.09906096]])

In [144]:
gs_svr.score(normed_test_data,test_labels)

-0.9234546414988912

In [145]:
# Evaluate the regressor
mse_one = mean_squared_error(test_labels.iloc[:,0], example_result[:,0])
mse_two = mean_squared_error(test_labels.iloc[:,1], example_result[:,1])
print(f'MSE for first regressor: {mse_one} - second regressor: {mse_two}')
mae_one = mean_absolute_error(test_labels.iloc[:,0], example_result[:,0])
mae_two = mean_absolute_error(test_labels.iloc[:,1], example_result[:,1])
print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two}')

MSE for first regressor: 1.6060195677087448 - second regressor: 0.13964789527320337
MAE for first regressor: 1.2478513015271417 - second regressor: 0.3377847436017212


In [146]:
index = 0
correct = 0
incorrect = 0
total = 0
for i in example_result:
    temp = np.array(example_result[index])
    predicted = temp.argsort()[-5:]
    actual = np.argmax(test_labels.iloc[index])
    if actual in predicted:
        correct+=1
    else:
        incorrect+=1
    total+=1
    index+=1
print(correct/total)
# np.argmax(example_result[1])

0.3766404199475066


12/12 [==============================] - 0s 797us/step - loss: 2.5287 - accuracy: 0.1181
Accuracy    : 0.12


12/12 [==============================] - 0s 668us/step - loss: 2.4720 - accuracy: 0.1732
Accuracy    : 0.17
